In [20]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from random import randint

from utility import *

In [45]:
@jit(nopython=True)
def neighbors_clustering(sol, points, K):
    neighbors = []

    q = int(len(sol)/5)

    choices = np.arange(len(sol))

    orig_val = squared_inner_distance(sol, points, K)
    centroids = calc_centroids(sol, points, K)

    for i in range(q):
        choice = np.random.randint(len(choices))
        choices = np.delete(choices, choice)
        for k in range(K):
            new_sol = sol.copy()
            if(k != new_sol[choice]):
                new_sol[choice] = k
                new_val = orig_val - np.linalg.norm(points[choice]-centroids[sol[choice]]) + np.linalg.norm(points[choice]-centroids[k])
                neighbors.append((new_sol,new_val))


    choices1 = np.arange(len(sol))
    choices2 = np.arange(len(sol))
    for i in range(q**2):
        choice1 = np.random.randint(len(choices1))
        choices1 = np.delete(choices1, choice1)
        choice2 = np.random.randint(len(choices2))
        choices2 = np.delete(choices2, choice2)

        new_sol = sol.copy()
        new_sol[choice1], new_sol[choice2] = new_sol[choice2], new_sol[choice1]

        new_val = orig_val - np.linalg.norm(points[choice1]-centroids[sol[choice1]]) - np.linalg.norm(points[choice2]-centroids[sol[choice2]]) + np.linalg.norm(points[choice1]-centroids[sol[choice2]]) + np.linalg.norm(points[choice2]-centroids[sol[choice1]])
        neighbors.append((new_sol,new_val))
        
    return neighbors


@jit(nopython=True)
def local_search(base_sol, evaluate, find_neighbourhood, points, K):
    old_sol = base_sol
    base_val = evaluate(old_sol, points, K)
    iter = 1
    same_sol = 0

    while True:
        neighbourhood = find_neighbourhood(old_sol, points, K)
        best_val = evaluate(old_sol, points , K)
        best_sol = old_sol

        print("Iteration number:", iter, "Valore percentuale:", best_val/base_val*100, "%")

        for sol,val in neighbourhood:
            if(val < best_val):
                best_val = val
                best_sol = sol
        
        if(best_sol is old_sol):
            same_sol = same_sol + 1
            if(same_sol == 100):
                break
        else:
            same_sol = 0
            old_sol = best_sol

        iter = iter+1
    return old_sol

In [46]:
points = load_points('C:/Users/franc/Desktop/prova.txt')
K = 15
N = len(points)

sol = np.random.randint(K, size = N) #soluzione iniziale
#sol = create_initial_sol(points, K)
sol = local_search(sol, squared_inner_distance, neighbors_clustering, points, K)

print("{:.5E}".format(squared_inner_distance(sol, points, K)))

Iteration number: 1 Valore percentuale: 100.0 %
Iteration number: 2 Valore percentuale: 99.9953231323872 %
Iteration number: 3 Valore percentuale: 99.98222473124149 %
Iteration number: 4 Valore percentuale: 99.97230886867914 %
Iteration number: 5 Valore percentuale: 99.96510254283064 %
Iteration number: 6 Valore percentuale: 99.949440577551 %
Iteration number: 7 Valore percentuale: 99.93358535094637 %
Iteration number: 8 Valore percentuale: 99.92383754220864 %
Iteration number: 9 Valore percentuale: 99.9057713446373 %
Iteration number: 10 Valore percentuale: 99.8923249700848 %
Iteration number: 11 Valore percentuale: 99.87893607804577 %
Iteration number: 12 Valore percentuale: 99.86257731041526 %
Iteration number: 13 Valore percentuale: 99.84916999854725 %
Iteration number: 14 Valore percentuale: 99.83280848642706 %
Iteration number: 15 Valore percentuale: 99.82040795502031 %
Iteration number: 16 Valore percentuale: 99.80692780093304 %
Iteration number: 17 Valore percentuale: 99.793543

In [12]:
print("{:.5E}".format(squared_inner_distance(sol, points, K)))

2.48744E+13
